# Beyond the basics of similarity search


<center>
<img src="https://flickrcode.files.wordpress.com/2017/03/retrieval.png">
</center>


<font size="-1">source: http://code.flickr.net</font>

# Contents

- Practical applications
- Similarity and metrics
- Brute-force method
- Faster (approximate) methods:
   - HNSW
   - FAISS
   - ANNOY
- Demo and code
- Summary
- Where to go next

# About me
### Physics -> Data Analysis -> Software Engineering -> Machine Learning

# Practical applications

- Classification
- Recommender systems
- Image retrieval engine
- Contextual advertising
- etc

|_ | _ |
:-------------------------:|:-------------------------:
<center>
<img src="images/recommender_systems.png" style="width: 400px;"/>|<img src="images/knn_classification.webp" alt="" style="width: 400px;"/>
</center>

<font size="-1">sources: 
    www.towardsdatascience.com;
    www.datacamp.com
</font>



# What does it mean to be "similar"?

<br>
<br>
<center>
<img src="https://www.rover.com/blog/wp-content/uploads/2016/12/chihuahua-or-muffin-food-that-looks-like-dogs.jpg">
</center>



- Euclidean distance (aka L2 norm)
- Manhattan distance (aka L1 norm, aka Taxicab norm)
- Hamming distance (aka overlap metric)
- Chebyshev distance
- Levenshtein distance
- Cosine similarity*
- Bregman divergence*
- and many, many more

There are also domain/dataset specific custom metrics: see **metric learning** or **similarity learning**

# Brute-force method


<br>
<br>
<center>
<img src="https://media.giphy.com/media/k9kKWKaUEDBM4/giphy.gif">
</center>


Given a query data point:
- Iterate through all points in the dataset
- Compute distances to each one wrt the query point
- Sort all points by distance
- Return top K points from the dataset

# Faster methods

- Hierarchical Navigable Small World (HNSW)
- FAISS
- ANNOY

### In essence: 
- Build a data structure in a clever way so that we can only search though a small subset of all points thus speeding up querying.
- Even better if we can reduce dimensionality of the data when doing this

# HNSW

- NSW algorithm:
   - small-world graph: social networks, networks of brain neurons, website navigation, etc
   - most `nodes` are not neighbors
   - can reach one node to another random node in a small number of steps
   
<center>
<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0306437913001300-gr1.jpg">
</center>


Layers (hierarchy)

<center>
<img src="https://image.slidesharecdn.com/converted20180822torosn2nickkimfinal-181016013429/95/toros-n2-lightweight-approximate-nearest-neighbor-library-69-638.jpg?cb=1539653734">
</center>


<font size="-1">source: 
    https://arxiv.org/ftp/arxiv/papers/1603/1603.09320.pdf
</font>

- Reduce likelihood of local minimum
- Separate into layers based on link length
- Traverse each layer until local minimum, start in the next layer from the last point

# FAISS

ADC: asymmetric distance computation
 - every data point is approximated by the centroid of the cluster it belongs to: $ y \approx q(y) $
 - quick search to find a closest cluster center, then more refined search within the found cluster

<center>
<img src="images/adc.png">
</center>

<font size="-1">source: 
    https://www.computer.org/csdl/trans/tp/2011/01/ttp2011010117.html
</font>

IVF: inverted file
   - reverse mapping: from `{vector: centroid}` to `{centroid: list-of-vectors}`
   
$vec_i -> q_1, vec_j -> q_2, vec_k -> q_2, vec_m -> q_1$

becomes

$q_1 -> [vec_i, vec_m], q_2 -> [vec_j, vec_k]$

PQ: product quantizer
   - quantize remainder vectors: $r(y) = y - q(y)$

<center>
<img src="http://mccormickml.com/assets/ProductQuantizer/compression.png">
</center>

<font size="-1">source: 
    www.inria.fr
</font>


Combine everything together:

<center>
<img src="https://www.researchgate.net/profile/Herve_Jegou/publication/47815472/figure/fig3/AS:339497498759180@1457953919644/Overview-of-the-inverted-file-with-asymmetric-distance-computation-IVFADC-indexing.png">
</center>


<font size="-1">source: 
    https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/
</font>

# ANNOY

- Pick two random points
- Split the space orthogonally to the "line" that connects them
- Rinse and repeat until every `leaf` has a "small" number of points

<center>
<img src="images/annoy_1.png">
</center>

<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

<center>
<img src="images/annoy_2.png">
</center>
<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

<center>
<img src="images/annoy_3.png">
</center>
<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

<center>
<img src="images/annoy_4.png">
</center>
<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

Additional tricks:
- explore the "wrong" side of the split
- build a forest of trees


<center>
<img src="images/annoy_5.png">
</center>
<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

![](https://erikbern.com/assets/2015/09/animated.gif)
<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

<center>
<img src="images/annoy_6.png">
</center>
<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

## ANNOY summary:

Preprocessing time:
- Build up a bunch of binary trees. For each tree, split all points recursively by random hyperplanes.

Query time:

- Insert the root of each tree into the priority queue
- Until we have `k` candidates, search all the trees using the priority queue
- Remove duplicate candidates
- Compute distances to candidates
- Sort candidates by distance
- Return the top ones

<font size="-1">source: 
    https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
</font>

# Example Dataset: GloVe vectors



In [1]:
# ! wget http://nlp.stanford.edu/data/glove.6B.zip
# ! mkdir -p glove_data
# ! unzip glove.6B.zip -d glove_data

In [2]:
! head -n1 glove_data/glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581


In [3]:
%%time 

import numpy as np

glove_vec_dict = {}
idx = 0
vectors = []
words = []
word2idx = {}
with open('glove_data/glove.6B.50d.txt', 'rb') as f:
    for line in f:
        splt = line.decode().split()
        word, vec = str(splt[0]), np.array(splt[1:]).astype(np.float)
        words.append(word)
        vectors.append(vec)
        word2idx[word] = idx        
        idx = idx + 1
    words = np.array(words)
    vectors = np.array(vectors).astype('float32')
    glove_word2vec = {words[idx]:vectors[idx] for idx in range(len(words))}

CPU times: user 10.6 s, sys: 1.16 s, total: 11.8 s
Wall time: 11.8 s


In [4]:
glove_word2vec['paris']

array([ 0.76989 ,  1.181   , -1.1299  , -0.74725 , -0.5969  , -1.0518  ,
       -0.46552 ,  0.27009 , -0.99243 , -0.04864 ,  0.28642 , -0.75261 ,
       -1.0566  , -0.19205 ,  0.572   , -0.24391 , -0.36054 , -0.70876 ,
       -0.91951 , -0.27024 ,  1.5131  ,  1.0313  , -0.55713 ,  0.52952 ,
       -0.71494 , -1.0949  , -0.60565 ,  0.31329 , -0.44488 ,  0.55915 ,
        2.1429  ,  0.43389 , -0.5529  , -0.24261 , -0.43679 , -0.96014 ,
        0.25828 ,  0.79385 ,  0.37132 ,  0.49623 ,  0.84359 , -0.25875 ,
        1.5616  , -1.1199  ,  0.091676,  0.076675, -0.45084 , -0.86104 ,
        0.97599 , -0.35615 ], dtype=float32)

In [5]:
vec = glove_word2vec['paris'] - glove_word2vec['france'] + glove_word2vec['japan']
vec = vec[None,:]
vec

array([[-0.21320996,  0.74222   ,  0.23972002,  0.99407   ,  0.51400006,
        -0.65760994, -0.302892  , -0.02410699, -1.55212   ,  0.04149997,
         0.30327994, -0.27375007, -0.04763797,  1.14008   ,  0.7124    ,
         0.98521996, -1.076494  ,  1.0097799 , -0.84392   ,  0.9916    ,
         2.6864662 , -0.03069007, -1.0861499 , -0.36579004,  0.8013    ,
        -1.4782999 , -0.04848   , -0.04828   , -0.49153   , -0.94323003,
         2.5423    ,  0.30223998,  1.14363   , -0.11400002, -0.48376995,
        -0.52109   ,  0.36661   , -0.271622  , -0.19778991,  0.20547998,
        -1.8284099 , -1.0849429 ,  0.61479986, -0.62539995,  1.078614  ,
        -0.21063498, -0.0346    ,  0.30837   ,  0.577846  , -0.17106003]],
      dtype=float32)

In [6]:
import faiss
import nmslib
from annoy import AnnoyIndex
from sklearn.neighbors import NearestNeighbors

In [7]:
%%time
index_brute = NearestNeighbors(n_neighbors=10,
                        algorithm='brute',
                        n_jobs=-1).fit(vectors)

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 12.5 ms


In [8]:
%%time
index_hnsw = nmslib.init(method='hnsw', space='l2')
index_hnsw.addDataPointBatch(vectors)
index_hnsw.createIndex({}, print_progress=True)

CPU times: user 6min 48s, sys: 2.53 s, total: 6min 51s
Wall time: 38.4 s


In [9]:
%%time
d = vec.shape[1]
nlist = 100
m = 5
k = 4
quantizer = faiss.IndexFlatL2(d)
index_faiss = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
index_faiss.train(vectors)
index_faiss.add(vectors)

CPU times: user 24.4 s, sys: 469 ms, total: 24.9 s
Wall time: 2.31 s


In [10]:
%%time
index_annoy = AnnoyIndex(vectors.shape[1], metric='euclidean')
for i, v in enumerate(vectors):
    index_annoy.add_item(i, v)
index_annoy.build(n_trees=20)

CPU times: user 11 s, sys: 406 ms, total: 11.5 s
Wall time: 9.46 s


True

In [11]:
def print_neighbors(vec, n_neighbors, index_name):
    if index_name == 'brute':
        idx_k_nbrs = index_brute.kneighbors(vec,
                                 n_neighbors=n_neighbors,
                                 return_distance=False)[0]
        for idx in idx_k_nbrs:
            print(words[idx])
    elif index_name == 'hnsw':
        idx_k_nbrs, _  = index_hnsw.knnQuery(vec, n_neighbors)
        for idx in idx_k_nbrs:
            print(words[idx])
    elif index_name == 'faiss':
        _, idx_k_nbrs = index_faiss.search(vec, n_neighbors)
        for idx in idx_k_nbrs[0]:
            print(words[idx])
    elif index_name == 'annoy':
        idx_k_nbrs = index_annoy.get_nns_by_vector(vec[0], n_neighbors)
        for idx in idx_k_nbrs:
            print(words[idx])

In [12]:
%time print_neighbors(vec=vec, n_neighbors=10, index_name='brute')

tokyo
shanghai
japan
osaka
japanese
beijing
seoul
singapore
kong
taipei
CPU times: user 312 ms, sys: 625 ms, total: 938 ms
Wall time: 111 ms


In [13]:
%time print_neighbors(vec=vec, n_neighbors=10, index_name='hnsw')

tokyo
shanghai
japan
osaka
japanese
beijing
seoul
singapore
kong
taipei
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.29 ms


In [14]:
%time print_neighbors(vec=vec, n_neighbors=10, index_name='faiss')

tokyo
japan
shanghai
athens
yen
osaka
morning
kong
hong
helsinki
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.28 ms


In [15]:
%time print_neighbors(vec=vec, n_neighbors=10, index_name='annoy')

tokyo
shanghai
japan
japanese
beijing
seoul
singapore
kong
taipei
hong
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.23 ms


# Summary:

- The underlying idea the same: 
    - only check a subset of all points; every algorithm has its own way of selecting the "best" subset
    - if possible, reduce dimensionality 
- There is no one "best" algorithm - selection of an algorithm will depend on:
    - processing time restrictions
    - precision/recall requirements
    - dataset distribution/structure
    - frequency of querying
    - frequency of inserting
    - hardware

# Interesting tools:
- http://ann-benchmarks.com/index.html - industry standard benchmarking tool for ANN
- https://euclidesdb.readthedocs.io/en/latest - cool project for performing similarity search in features extracted by Deep Learning models

# Blog posts:

- https://erikbern.com/2015/09/24/nearest-neighbor-methods-vector-models-part-1.html
- https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html
- https://www.benfrederickson.com/approximate-nearest-neighbours-for-recommender-systems/
- http://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/
- http://mccormickml.com/2017/10/22/product-quantizer-tutorial-part-2/